In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix# Assuming x is a list of text documents and y is a binary label (0 or 1)
# You need to convert text data into numerical features using CountVectorizer
from sklearn.linear_model import LogisticRegression
import pickle
n_scenes = 13


In [13]:
#extract RL features
model_name = 'RLnv'
feature = 'q_values'
for n in tqdm(range(n_scenes)):
    outputs = torch.stack([results[model_name][n][i][feature] for i in results[model_name][n].keys()]).cpu().detach()
    with open(model_name+feature+'/'+str(n)+'.pickle', 'wb') as f:
            pickle.dump(outputs,f)

100%|█████████████████████████████████████████| 13/13 [00:00<00:00, 1048.52it/s]


In [10]:
#extract RL features


    
distances_Y = []
q_values_X = []
n_scenes = 13

distances_Y_test = []
q_values_X_test = []
train = np.array([ 4,  9,  8,  5,  6,  3,  0,  1, 12, 10])
model_name = 'RLnv'
feature = 'q_values'
for n in tqdm(range(n_scenes)):
    with open('scene_'+str(n)+'.pickle', 'rb') as f:
            data,image_dict = pickle.load(f)
    distances = np.array([image_dict[i]['distance'] for i in image_dict.keys()])
    q_values = torch.stack([results[model_name][n][i][feature] for i in results[model_name][n].keys()]).cpu().detach()
    with open('RLnvq_values/'+str(n)+'.pickle', 'wb') as f:
            pickle.dump(outputs,f)


100%|███████████████████████████████████████████| 13/13 [00:17<00:00,  1.35s/it]


In [12]:
#save distances


n_scenes = 13

for n in tqdm(range(n_scenes)):
    with open('scene_'+str(n)+'.pickle', 'rb') as f:
            data,image_dict = pickle.load(f)
    distances = np.array([image_dict[i]['distance'] for i in image_dict.keys()])
    with open('distances/'+str(n)+'.pickle', 'wb') as f:
            pickle.dump(distances,f)


100%|███████████████████████████████████████████| 13/13 [00:06<00:00,  2.03it/s]


In [ ]:
with open('dino_features.pickle', 'rb') as f:
                X,Y,Y_reg,X_test,Y_reg_test, Y_test = pickle.load(f)

In [1]:
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
import requests
import torch 
import numpy as np
import pickle
from tqdm import tqdm

device = 'cuda'
# Assuming you have a processor that can convert logits to predictions
# (e.g., converting logits to segmentation mask using argmax)
processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b2-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b2-finetuned-ade-512-512").to(device)

distances_Y = []
q_values_X = torch.tensor([])
n_scenes = 13

distances_Y_test = []
q_values_X_test = torch.tensor([])
train = np.array([ 4,  9,  8,  5,  6,  3,  0,  1, 12, 10])
model_name = 'RLfloor'
for n in range(n_scenes):
    with open('scene_'+str(n)+'.pickle', 'rb') as f:
            data,image_dict = pickle.load(f)
    distances = np.array([image_dict[i]['distance'] for i in image_dict.keys()])
    
    images = np.array([image_dict[i]['img'] for i in image_dict.keys()])
    inputs = [processor(images=torch.tensor(image).unsqueeze(0), return_tensors="pt") for image in tqdm(images)]
    for input in tqdm(inputs):
        input['pixel_values'] = input['pixel_values'].to('cuda')
    with torch.no_grad():
        outputs = torch.stack([model(**input,output_hidden_states=True).hidden_states[3].flatten().cpu().detach() for input in tqdm(inputs)])
    with open('segformer_features/'+str(n)+'.pickle', 'wb') as f:
                pickle.dump(outputs,f)
    if n in train:
        print('listing...')
        
        q_values_X = torch.cat((q_values_X, outputs))
        distances_Y.extend(distances)
    else:
        q_values_X_test = torch.cat((q_values_X_test, outputs))
        distances_Y_test.extend(distances)





/home/isarbout/.pyenv/versions/3.10.12/envs/unity/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-02-28 23:33:33.077116: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 23:33:33.078842: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 23:33:33.097019: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 23:33:33.097032: E tensorflow/compiler/xla/stream_executor/cuda

listing...


100%|██████████████████████████████████████| 1536/1536 [00:13<00:00, 115.28it/s]


listing...


100%|██████████████████████████████████████| 1500/1500 [00:12<00:00, 119.04it/s]


...

listing...


100%|██████████████████████████████████████| 1548/1548 [00:13<00:00, 118.28it/s]


listing...


100%|██████████████████████████████████████| 1488/1488 [00:12<00:00, 118.59it/s]


listing...


In [4]:
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
#compute dino features
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base').to('cuda')

distances_Y = []
q_values_X = torch.tensor([])
n_scenes = 13

distances_Y_test = []
q_values_X_test = torch.tensor([])
train = np.array([ 4,  9,  8,  5,  6,  3,  0,  1, 12, 10])
for n in range(n_scenes):
    with open('scene_'+str(n)+'.pickle', 'rb') as f:
            data,image_dict = pickle.load(f)
    distances = np.array([image_dict[i]['distance'] for i in image_dict.keys()])
    
    images = np.array([image_dict[i]['img'].resize((224,224)) for i in image_dict.keys()])
    inputs = [processor(images=torch.tensor(image).unsqueeze(0), return_tensors="pt") for image in tqdm(images)]
    for input in tqdm(inputs):
        input['pixel_values'] = input['pixel_values'].to('cuda')
    with torch.no_grad():
        outputs = torch.stack([model(**input).last_hidden_state.flatten().cpu().detach() for input in tqdm(inputs)])
    with open('dinov2_features/'+str(n)+'.pickle', 'wb') as f:
                pickle.dump(outputs,f)



/home/isarbout/.pyenv/versions/3.10.12/envs/unity/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-02-28 23:51:22.102238: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-28 23:51:22.105061: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 23:51:22.128009: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-28 23:51:22.128031: E tensorflow/compiler/xla/stream_executor/cuda

In [5]:
import torch
from datasets import load_dataset
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification

dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b7")
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b7").to('cuda')

for n in range(n_scenes):
    with open('scene_'+str(n)+'.pickle', 'rb') as f:
            data,image_dict = pickle.load(f)
    distances = np.array([image_dict[i]['distance'] for i in image_dict.keys()])
    
    images = np.array([image_dict[i]['img'] for i in image_dict.keys()])
    inputs = [preprocessor(images=torch.tensor(image).unsqueeze(0), return_tensors="pt") for image in tqdm(images)]
    for input in tqdm(inputs):
        input['pixel_values'] = input['pixel_values'].to('cuda')
    with torch.no_grad():
        outputs = torch.stack([model(**input,output_hidden_states=True).hidden_states[55].flatten().cpu().detach() for input in tqdm(inputs)])
    with open('EfficientNetb7_features/'+str(n)+'.pickle', 'wb') as f:
                pickle.dump(outputs,f)



100%|███████████████████████████████████████| 1488/1488 [00:16<00:00, 92.62it/s]


In [10]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch


model = timm.create_model(
    'convnextv2_atto.fcmae',
    pretrained=True,
    num_classes=0,  # remove classifier nn.Linear
).to('cuda')
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

for n in range(n_scenes):
    with open('scene_'+str(n)+'.pickle', 'rb') as f:
            data,image_dict = pickle.load(f)
    distances = np.array([image_dict[i]['distance'] for i in image_dict.keys()])
    
    images = [image_dict[i]['img'] for i in image_dict.keys()]
    inputs = [transforms(image).to('cuda').unsqueeze(0) for image in tqdm(images)]
    with torch.no_grad():
        outputs = torch.stack([model.forward_features(input).flatten().cpu().detach() for input in tqdm(inputs)])
    with open('ConvNextv2_features/'+str(n)+'.pickle', 'wb') as f:
                pickle.dump(outputs,f)



100%|██████████████████████████████████████| 1488/1488 [00:03<00:00, 455.18it/s]
